In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as t
import requests
import json
import math
import pandas as pd
import datetime
import boto3
import os
from urllib.parse import urljoin
from datetime import date
from datetime import datetime

In [ ]:
DBATSECRETSCOPE = "Production-ELT"
DBPATKEY = "DB_API_Token"

host_token = dbutils.secrets.get(DBATSECRETSCOPE, DBPATKEY)

API_ENDPOINT = '/api/2.0/preview/sql/queries'

# Databricks API key to authenticate with the Databricks REST API
headers = {"Authorization": "Bearer " + host_token}

# configure the Databricks workspace URL
host_name = "<work-place-url>"

bucket = '<work-s3-bucket>'
folder = 'databricks-queries-settings-weekly/'

In [ ]:
def get_queries(api_url, token, page_size=25):
    all_queries = []
    page = 1
    total_pages = 1  # Initialize

    while page <= total_pages:
        params = {
            'page': page,
            'page_size': page_size
        }
        headers = {
            'Authorization': f'Bearer {token}'
        }
        response = requests.get(urljoin(host_name, api_url), headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            total_pages = math.ceil(data['count'] / page_size)  # Update total_pages based on response
            results = data['results']

            for query in results:
                if not query['is_draft']:
                    all_queries.append(query)

            page += 1
        else:
            # Handle errors
            print(f"Error: {response.status_code}")
            break

    return all_queries


# save queries to s3 bucket
def save_query_data_to_s3(bucket_name, user_name, data):
    s3 = boto3.client('s3')

    # Get the current timestamp in YYYY-MM-DD_HH-MM-SS-ms format
    now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S-%f')

    # Check if the user name is available, otherwise use a default folder name
    if user_name.lower() == "unavailable":
        folder_name = "unknown_users"
    else:
        # Format the folder name: lowercase and replace spaces with underscores
        folder_name = user_name.replace(" ", "_").lower()

    # Define the main folder name
    main_folder = 'databricks-queries-settings-weekly'

    # Construct the full key with the main folder, user folder, and file name including the timestamp
    key = f"{main_folder}/{folder_name}/query_data_{folder_name}_{now}.json"
    json_data = json.dumps(data, indent=4)

    try:
        s3.put_object(Bucket=bucket_name, Key=key, Body=json_data, ContentType='application/json')
        print(f"Data for user {user_name} successfully saved to {bucket_name}/{key}")
    except Exception as e:
        print(f"Failed to save data for user {user_name} to S3. Error: {e}")


In [ ]:
# save job setting to aws s3 bucket/folder
bucket = '<work-s3-bucket>'

queried_data = get_queries(API_ENDPOINT, host_token)
#save_query_data_to_s3(bucket_name, user_name, data)
for data in queried_data:
    name = data["user"]["name"]
    save_query_data_to_s3(bucket, name, queried_data)